In [ ]:
import requests # type: ignore
import numpy as np # type: ignore
import re
import json
import pandas as pd
import os

In [ ]:
NUM_COHORTS = 6 # 24
COHORT_SIZE = 4 # 8
NUM_ROUNDS = 3 # 75

MEMORY_DIR = "memory"
MEMORY_SIZE = 10
MODEL_NAME= "mistral"

In [ ]:
TREATMENTS = {
	0.6: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 42),
		('B', 'R'): (42, 0),
		('B', 'B'): (12, 12)
	},
	1: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 40),
		('B', 'R'): (40, 0),
		('B', 'B'): (20, 20)
	},
	2: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 35),
		('B', 'R'): (35, 0),
		('B', 'B'): (40, 40)
	}
}

In [ ]:
API_URL = "http://127.0.0.1:11434"
GENERATE_ENDPOINT = "/api/generate"
CHAT_ENDPOINT = "/api/chat"

In [ ]:
rand_gen = np.random.default_rng(seed=23)

In [ ]:
class Participant:
	def __init__(self, id):
		self.id = id
		self.memory = []
		self.payout = 0
		self.rounds = 0
		self.prompt = ""

	def __eq__(self, other):
		return self.id == other.id
	
	def set_prompt(self, prompt):
		self.prompt = prompt
	
	def update_memory(self, ctx):
		self.memory = [*ctx]
		if len(self.memory) >= MEMORY_SIZE:
			self.memory.pop(0)
		self.save_memory()
		
	def save_memory(self):
		with open(f"{MEMORY_DIR}/{self.id}.json", "wt") as file:
			file.write(json.dumps(self.memory))

In [ ]:

participant_ids = np.arange(101, NUM_COHORTS * COHORT_SIZE + 101, dtype=int)
rand_gen.shuffle(participant_ids)
participants = [Participant(id) for id in participant_ids]
cohorts = np.array_split(participants, NUM_COHORTS)

In [ ]:
split_cohorts = np.array_split(cohorts, len(TREATMENTS))
cohort_treatments = list(zip(split_cohorts, TREATMENTS.keys()))

In [ ]:
def prompt_model(prompt, context):
	messages = [
		*context,
		{
			"role": "user",
			"content": prompt
		}
	]
	data = {
    "model": MODEL_NAME,
		"messages": messages,
    "stream": False
	}
	res = requests.post(API_URL + CHAT_ENDPOINT, json=data).json()
	return [*messages, res['message']]

In [ ]:
def get_choice(response):
	matches = re.findall("{[A-Z]}", response)
	if len(matches) > 0:
		return matches[-1][1]
	else:
		matches = re.findall("[RB]", response)
		if len(matches) > 0:
			return matches[-1]
		else:
			return ""

In [ ]:
def generate_start_prompt(treatment):
	return f"You are an undergraduate student participating in a lab experiment. You play a game with an anonymous player in which you simultaneously make a choice. Your payoff depends on both choices. If you both pick R, you each get {TREATMENTS[treatment][('R','R')][0]}$. If you choose R while they choose B, you get {TREATMENTS[treatment][('R','B')][0]}$, and they get {TREATMENTS[treatment][('R','B')][1]}$. Similarly, if you pick B while they pick R, you get {TREATMENTS[treatment][('B','R')][0]}$, and they get {TREATMENTS[treatment][('B','R')][1]}$. If you both pick B, you each earn {TREATMENTS[treatment][('B','B')][0]}$. The game has {NUM_ROUNDS} rounds. What's your choice? Perform reasoning as a human player. Append your choice letter in curly brackets as a last character."

In [ ]:
def generate_cont_prompt(choice1, choice2, treatment):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. Please play the next round."

In [ ]:
def generate_end_prompt(choice1, choice2, treatment, payout):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. The game is now over. Your total payoff was {payout + TREATMENTS[treatment][(choice1, choice2)][0]}$."

In [ ]:
def save_results(filename, row):
	new_row = pd.DataFrame(row)
	if os.path.isfile(filename):
		data = pd.read_csv(filename)
		data = pd.concat([data, new_row])
		data.to_csv(filename, index=False)
	else:
		data = new_row
		data.to_csv(filename, index=False)

In [ ]:
for cohorts_for_treatment, treatment in cohort_treatments:
	for cohort in cohorts_for_treatment:
		for round in range(1, (NUM_COHORTS * COHORT_SIZE * NUM_ROUNDS // 2) + 1):
			participant, opponent = rand_gen.choice(cohort, size=2, replace=False)

			print(f"Treatment: {treatment} - Round {round}: {participant.id} vs {opponent.id}")

			if participant.rounds == 0:
				print(f"First round of participant {participant.id}")
				participant.set_prompt(generate_start_prompt(treatment))

			if opponent.rounds == 0:
				print(f"First round of participant {opponent.id}")
				opponent.set_prompt(generate_start_prompt(treatment))

			print(f"Prompting participant {participant.id}")
			ctx = prompt_model(participant.prompt, participant.memory)
			choice1 = get_choice(ctx[-1]['content'])
			print(f"Got response for participant {participant.id} with choice: {choice1}")
			participant.update_memory(ctx)

			print(f"Prompting participant {opponent.id}")
			ctx = prompt_model(opponent.prompt, opponent.memory)
			choice2 = get_choice(ctx[-1]['content'])
			print(f"Got response for participant {opponent.id} with choice: {choice2}")
			opponent.update_memory(ctx)

			participant.rounds += 1
			opponent.rounds += 1

			participant.payout = TREATMENTS[treatment][(choice1, choice2)][0]
			opponent.payout = TREATMENTS[treatment][(choice1, choice2)][1]

			print(f"Saving results to file")
			save_results('experiment.csv', {'treatment': [treatment], 'round': [round], 'player1_id': [participant.id], 'player2_id': [opponent.id], 'player1_choice': [choice1], 'player2_choice': [choice2], 'player1_payout': [participant.payout], 'player2_payout': [opponent.payout]})
			print(f"Saved results to file")

			if participant.rounds == NUM_ROUNDS:
				participant.set_prompt(generate_end_prompt(choice1, choice2, treatment, participant.payout))
			else:
				participant.set_prompt(generate_cont_prompt(choice1, choice2, treatment))

			if opponent.rounds == NUM_ROUNDS:
				opponent.set_prompt(generate_end_prompt(choice1, choice2, treatment, opponent.payout))
			else:
				opponent.set_prompt(generate_cont_prompt(choice2, choice1, treatment))